In [ ]:
import numpy as np

# Supongamos que tenemos una matriz de puntos de tamaño (N, 3)

npuntos = 20000
puntos = np.random.rand(npuntos, 3)

# Definimos el tamaño del voxel
tam_voxel = 0.1

# Calculamos el número de voxels en cada dimensión
n_voxels = np.ceil((np.max(puntos, axis=0) - np.min(puntos, axis=0)) / tam_voxel).astype(int)

# Creamos una matriz de índices de voxels para cada punto
indices_voxels = np.floor((puntos - np.min(puntos, axis=0)) / tam_voxel).astype(int)

# Creamos una matriz de frecuencias de voxels
frec_voxels = np.zeros((n_voxels[0], n_voxels[1], n_voxels[2]), dtype=int)

# Recorremos cada voxel y contamos la frecuencia de puntos que caen dentro de él
for i in range(n_voxels[0]):
    for j in range(n_voxels[1]):
        for k in range(n_voxels[2]):
            mascara = (indices_voxels[:, 0] == i) & (indices_voxels[:, 1] == j) & (indices_voxels[:, 2] == k)
            frec_voxels[i, j, k] = np.sum(mascara)

# Creamos una matriz de etiquetas para cada punto
etiquetas = np.zeros(npuntos, dtype=int)

# Recorremos cada voxel y asignamos la etiqueta más frecuente a los puntos que caen dentro de él
for i in range(n_voxels[0]):
    for j in range(n_voxels[1]):
        for k in range(n_voxels[2]):
            mascara = (indices_voxels[:, 0] == i) & (indices_voxels[:, 1] == j) & (indices_voxels[:, 2] == k)
            puntos_voxel = puntos[mascara]
            if puntos_voxel.shape[0] > 0:
                etiquetas[mascara] = np.argmax(np.bincount(puntos_voxel[:, -1].astype(int)))

# Creamos una matriz de frecuencias de etiquetas
frec_etiquetas = np.bincount(etiquetas)

# Seleccionamos los 16384 puntos más representativos
puntos_sel = np.zeros((16384, 3))
for i in range(16384):
    voxel_mas_frecuente = np.unravel_index(np.argmax(frec_voxels), frec_voxels.shape)
    mascara = (indices_voxels[:, 0] == voxel_mas_frecuente[0]) & (indices_voxels[:, 1] == voxel_mas_frecuente[1]) & (indices_voxels[:, 2] == voxel_mas_frecuente[2])
    puntos_voxel = puntos

    if puntos_voxel.shape[0] > 0:
        etiquetas_voxel = etiquetas[mascara]
        etiqueta_mas_frecuente = np.argmax(np.bincount(etiquetas_voxel))
        puntos_mas_representativos = puntos_voxel[etiquetas_voxel == etiqueta_mas_frecuente][:, :-1]
        punto_mas_cercano = puntos_mas_representativos[np.argmin(np.sum(np.square(puntos_mas_representativos - puntos_sel[:i]), axis=1))]
        puntos_sel[i] = np.concatenate((punto_mas_cercano, np.array([etiqueta_mas_frecuente])))
        frec_voxels[voxel_mas_frecuente] = 0
    else:
        break


In [9]:
import numpy as np
from sklearn.cluster import KMeans

def downsample_point_cloud(point_cloud, num_points=1634, voxel_size=0.1):
    # Create voxel grid
    min_bounds = np.min(point_cloud[:, :3], axis=0)
    max_bounds = np.max(point_cloud[:, :3], axis=0)
    voxel_grid = np.zeros(np.ceil((max_bounds - min_bounds) / voxel_size).astype(int))
    for i in range(point_cloud.shape[0]):
        voxel_coords = np.floor((point_cloud[i, :3] - min_bounds) / voxel_size).astype(int)
        voxel_grid[tuple(voxel_coords)] += point_cloud[i, :3]
    voxel_centers = []
    for coords, voxel_sum in np.ndenumerate(voxel_grid):
        if voxel_sum.any():
            voxel_points = point_cloud[tuple(np.argwhere(np.all(point_cloud[:, :3] == voxel_sum, axis=1))[0]), :]
            if not np.isnan(voxel_points[:, 3:]).any():
                voxel_intensity = np.mean(voxel_points[:, 3])
                voxel_class = np.mean(voxel_points[:, 4])
                voxel_centers.append(np.concatenate((voxel_sum / np.sum(voxel_grid[coords]), [voxel_intensity, voxel_class])))
    voxel_centers = np.array(voxel_centers)

    # Select subset of points
    if voxel_centers.shape[0] <= num_points:
        return voxel_centers
    else:
        kmeans = KMeans(n_clusters=num_points)
        kmeans.fit(voxel_centers[:, :3])
        return np.concatenate((kmeans.cluster_centers_, voxel_centers[:, 3:]), axis=1)


In [79]:
import open3d as o3d
import numpy as np
from matplotlib import cm

# Define colors for each class
MisColores = {
    1: np.array([0.5,0.25,0]), #terreno
    2: np.array([0.1,0.6,0]),  #vegetación
    3: np.array([0.99,0,0]), #coche
    4: np.array([0.5,0.5,0.5]), #torre
    5: np.array([0.99,0,0.99]), #cable
    6: np.array([0.1,0.1,0.1]), #valla/muro
    7: np.array([0.99,0.99,0]), #farola
    8: np.array([0.15,0.09,0.9]), #edificio
}

#data = np.load('pruebas_las_cortados/section_431749.71_4598734.71.npy')

data = np.load('cortados/d2RCD-4-grafos-joint_153.npy')
data = np.load('nube_de_puntos.npy')
x = data[:, 0]
y = data[:, 1]
z = data[:, 2]
c = data[:, 4].astype(int)
r = data[:, 3]

points = data[:, 0:3]

# Assign colors to each point based on the corresponding value in the c array
colors = np.zeros_like(points)
for i, value in MisColores.items():
    mask = (c == i)
    colors[mask] = value

cloud = o3d.geometry.PointCloud()
cloud.points = o3d.utility.Vector3dVector(points)
cloud.colors = o3d.utility.Vector3dVector(colors)
o3d.visualization.draw_geometries([cloud])


print("Downsample the point cloud with a voxel of 0.05")
downpcd = cloud.farthest_point_down_sample(num_samples=16384)
o3d.visualization.draw_geometries([downpcd])
len(downpcd.points)


Downsample the point cloud with a voxel of 0.05


16384

In [82]:
import numpy as np
import open3d as o3d

# Load the point cloud from a numpy file
data = np.load('cortados/d2RCD-4-grafos-joint_153.npy')
points = data[:, :3]
colors = data[:, 4:]

# Create an Open3D point cloud object and assign colors
cloud = o3d.geometry.PointCloud()
cloud.points = o3d.utility.Vector3dVector(points)


# Downsample the point cloud using farthest point sampling
downpcd = cloud.farthest_point_down_sample(num_samples=16384)

# Save the downsampled point cloud as a numpy file
np.save('downsampled.npy', np.asarray(downpcd.points))

# Visualize the downsampled point cloud
o3d.visualization.draw_geometries([downpcd])



In [1]:
import numpy as np
import open3d as o3d

# Load the point cloud from a numpy file
data = np.load('nube_de_puntos.npy')
data = np.load('cortados/d2RCD-4-grafos-joint_153.npy')

# Extract x, y, z, c, and r from the data array
x = data[:, 0]
y = data[:, 1]
z = data[:, 2]
c = data[:, 4].astype(int)
r = data[:, 3]

# Create a new array with the coordinates, colors, and intensities
points_with_data = np.column_stack((x, y, z, c, r))

# Create an Open3D point cloud object
cloud = o3d.geometry.PointCloud()
cloud.points = o3d.utility.Vector3dVector(points_with_data[:, :3])
#cloud.colors = o3d.utility.Vector3dVector(points_with_data[:, 3:6])

# Downsample the point cloud using farthest point sampling
downpcd = cloud.farthest_point_down_sample(num_samples=16384)

# Save the downsampled point cloud as a numpy file
np.save('downsampled.npy', downpcd.points)

# Save the downsampled point cloud with its color and intensity values
np.save('downsampled_with_data.npy', points_with_data[downpcd.indicies])

# Visualize the downsampled point cloud
o3d.visualization.draw_geometries([downpcd])


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


AttributeError: 'open3d.cpu.pybind.geometry.PointCloud' object has no attribute 'indicies'

In [ ]:
data = np.load('cortados/d2RCD-4-grafos-joint_049.npy')

MisColores = {
    1: np.array([0.5,0.25,0]), #terreno
    2: np.array([0.1,0.6,0]),  #vegetación
    3: np.array([0.99,0,0]), #coche
    4: np.array([0.5,0.5,0.5]), #torre
    5: np.array([0.99,0,0.99]), #cable
    6: np.array([0.1,0.1,0.1]), #valla/muro
    7: np.array([0.99,0.99,0]), #farola
    8: np.array([0.15,0.09,0.9]), #edificio
}

x = data[:, 0]
y = data[:, 1]
z = data[:, 2]
c = data[:, 4].astype(int)
r = data[:, 3]

points = data[:, 0:3]

point_cloud = data[:, 0:5]

In [17]:



    print("Downsample the point cloud with a voxel of 0.05")
    downpcd = pcd.voxel_down_sample(voxel_size=0.05)
    o3d.visualization.draw_geometries([downpcd])

Load a ply point cloud, print it, and render it
[Open3D WARNING] Read geometry::PointCloud failed: unknown file extension for cortados/d2RCD-4-grafos-joint_049.npy (format: auto).
PointCloud with 0 points.
[]
Downsample the point cloud with a voxel of 0.05
Recompute the normal of the downsampled point cloud
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
Print a normal vector of the 0th point


IndexError: 